In [5]:
import geopandas as gpd
import pandas as pd
import warnings
from shapely.geometry import Point

# Apaga todas las advertencias (warnings)
warnings.filterwarnings("ignore")

def get_path(source ,feature, mode='caminata'):
    return f'/app/data/{source}/red_escenario_futuro/{mode}/{feature}_conce'
def get_input_nodes():
    return get_path('input', 'nodes')
def get_input_edges():
    return get_path('input', 'edges')
def get_output_nodes():
    return get_path('output', 'nodes')
def get_output_edges():
    return get_path('output', 'edges')


In [2]:
edges = gpd.read_file(get_input_edges())
nodes = gpd.read_file(get_input_nodes())

Procesamiento de los nodos

In [3]:
nodes['temp_id'] = nodes.index
nodes['str_id'] = nodes['temp_id'].astype(str)
nodes.loc[nodes['osmid']==0, 'osmid'] = '992023' + nodes['str_id']
nodes.drop(columns=['temp_id', 'str_id'], inplace=True)

nodes.x = nodes.geometry.x
nodes.y = nodes.geometry.y
net_nodes = nodes.copy()
net_nodes.drop(columns=['x', 'y'], inplace=True)

Procesamiento de los vertices

In [7]:
# Seleccionamos solos los nuevos vertices (osmid que empieza con 2023)
edges_mask = edges['osmid'].astype(str).str.startswith("2023")
edges_cols = ['osmid', 'length', 'geometry']
new_edges = edges.loc[edges_mask, edges_cols]

Le asignamos un nodo de esquina a los vertices nuevos

In [8]:
# Añade una nueva columna llamada "begin" que contiene el primer punto de cada línea
new_edges['begin'] = new_edges['geometry'].apply(lambda geom: Point(geom.coords[0]))

# Añade una nueva columna llamada "end" que contiene el último punto de cada línea
new_edges['end'] = new_edges['geometry'].apply(lambda geom: Point(geom.coords[-1]))

# Función para encontrar el punto más cercano en el GeoDataFrame "nodes"
def find_nearest_node(point):
    nearest = nodes['geometry'].apply(lambda x: point.distance(x)).idxmin()
    return nodes.loc[nearest, 'osmid']

# Aplicamos la función para encontrar el "source" y "destination" para cada línea en "new_edges"
new_edges['u'] = new_edges['begin'].apply(find_nearest_node)
new_edges['v'] = new_edges['end'].apply(find_nearest_node)

In [9]:
edges_cols = ['u', 'v', 'osmid', 'length', 'geometry']
old_edges = edges.loc[~edges_mask, edges_cols]
new_edges = new_edges[edges_cols]
net_edges = pd.concat([old_edges, new_edges])

In [10]:
def format_net_nodes(nodes_df):
    nodes = pd.DataFrame(
        {
            'osmid': nodes_df['osmid'].astype(int),
            'lat' : nodes_df.geometry.y.astype(float),
            'lon' : nodes_df.geometry.x.astype(float),
            'y' : nodes_df.geometry.y.astype(float),
            'x' : nodes_df.geometry.x.astype(float),
        }
    )
    nodes['id'] = nodes['osmid'].values

    nodes = gpd.GeoDataFrame(data=nodes, geometry=nodes_df.geometry)
    nodes.set_index('osmid', inplace=True)
    return nodes

nodes = format_net_nodes(net_nodes)

In [22]:
def format_net_edges(edges_df):
    edges = pd.DataFrame(
        {
            'u': edges_df['u'].astype(int),
            'v': edges_df['v'].astype(int),
            'from': edges_df['u'].astype(int),
            'to': edges_df['v'].astype(int),
            'osmid': edges_df['osmid'].astype(int),
            'length': edges_df['length'].astype(float)
        }
    )
    edges['key'] = 0
    edges['key'] = edges['key'].astype(int)
    edges = gpd.GeoDataFrame(data=edges, geometry=edges_df.geometry)
    edges.set_index(['u', 'v', 'key'], inplace=True)
    return edges

edges = format_net_edges(net_edges)

In [23]:
mask = (~edges['to'].isin(nodes['id']))|(~edges['from'].isin(nodes['id']))
edges_ok = edges[~mask]
edges_to_fix = edges[mask].reset_index(drop=True)

In [25]:
# Añade una nueva columna llamada "begin" que contiene el primer punto de cada línea
edges_to_fix['begin'] = edges_to_fix['geometry'].apply(lambda geom: Point(geom.coords[0]))

# Añade una nueva columna llamada "end" que contiene el último punto de cada línea
edges_to_fix['end'] = edges_to_fix['geometry'].apply(lambda geom: Point(geom.coords[-1]))

# Función para encontrar el punto más cercano en el GeoDataFrame "nodes"
def find_nearest_node(point):
    nearest = nodes['geometry'].apply(lambda x: point.distance(x)).idxmin()
    return nodes.loc[nearest, 'id']

# Aplicamos la función para encontrar el "source" y "destination" para cada línea en "new_edges"
edges_to_fix['u'] = edges_to_fix['begin'].apply(find_nearest_node)
edges_to_fix['v'] = edges_to_fix['end'].apply(find_nearest_node)

In [26]:
edges_fixed = format_net_edges(edges_to_fix)

In [28]:
out_edges = pd.concat([edges_ok, edges_fixed])

In [30]:
out_nodes = nodes.copy()

In [31]:
out_nodes.to_file(get_output_nodes())
out_edges.to_file(get_output_edges())